Proyecto Telecom X - Etapa 2 - Machine Learning
---

# Predicción de Evasión (Churn) de clientes.

## Índice

# 1. Descripción del proyecto

# 2. Importación de librerías y configuraciones

## Importación de librerías

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import requests
from pathlib import Path

from sklearn.model_selection import train_test_split as tts
from sklearn.model_selection import KFold, cross_validate
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.compose import make_column_transformer
from sklearn.dummy import DummyClassifier
from sklearn.ensemble import RandomForestClassifier

## Paths

In [2]:
PROJECT_PATH = 'C:/Users/Ignacio/JupyterScripts/ONE/challenge_telecomX_parte_2'

src = Path('src')
DATA_PATH = PROJECT_PATH / src

img = Path('img')
IMG_PATH = PROJECT_PATH / img

models = Path('models')
MODELS_PATH = PROJECT_PATH / models

reports = Path('Reports')
REPORTS_PATH = PROJECT_PATH / reports


dirs = [DATA_PATH, IMG_PATH, MODELS_PATH, REPORTS_PATH]


for directory in dirs:
    if not directory.exists():
        directory.mkdir(parents=True)
        print(f'Directoryo creado:\n{directory}\n')
    else:
        print(f'El directorio ya existe:\n{directory}\n')

Directoryo creado:
C:\Users\Ignacio\JupyterScripts\ONE\challenge_telecomX_parte_2\src

Directoryo creado:
C:\Users\Ignacio\JupyterScripts\ONE\challenge_telecomX_parte_2\img

Directoryo creado:
C:\Users\Ignacio\JupyterScripts\ONE\challenge_telecomX_parte_2\models

Directoryo creado:
C:\Users\Ignacio\JupyterScripts\ONE\challenge_telecomX_parte_2\Reports



## Funciones

# 3. Preprocesamiento de datos

# 4. Modelado de datos

## Baseline model